In [1]:
# Built-in modules
import subprocess

# Third-party modules
import py3Dmol
from pymol import cmd 
from ipywidgets import interact, IntSlider

# Custom modules
from dock import AutoDockVina

MOL = '../molecules'

In [ ]:
# Prepare Breve for docking

# Load
breveID = 'D4BMM7'
url = f"https://alphafold.ebi.ac.uk/files/AF-{breveID}-F1-model_v4.pdb"
cmd.load(url, "breve")
cmd.save(f"{MOL}/{breveID}.pdb", "breve")
# Align with 8IBT -> so far it's not working, had to do it manually in PyMol
cmd.load(f"{MOL}/8ibt.pdb", "8ibt")
cmd.align("breve", "8IBT", object="alignment",cutoff=1000, cycles=0, quiet=0)
cmd.save(f"{MOL}/{breveID}_aligned.pdb", "alignment")
# PDB -> PDBQT
command = f'mk_prepare_receptor.py -i {MOL}/{breveID}_aligned.pdb -o {MOL}/{breveID}_aligned -p'
subprocess.run(command, shell=True, capture_output=False)

 Match: read scoring matrix.
 Match: assigning 710 x 694 pairwise scores.
 MatchAlign: aligning residues (710 vs 694)...
 MatchAlign: score 1882.000
 ExecutiveAlign: 4697 atoms aligned.
 Executive: RMSD =    2.531 (4697 to 4697 atoms)
 Executive: object "alignment" created.


@> 5578 atoms and 1 coordinate set(s) were parsed in 0.05s.



Files written:
../molecules/D4BMM7_aligned.pdbqt <-- static (i.e., rigid) receptor input file


CompletedProcess(args='mk_prepare_receptor.py -i ../molecules/D4BMM7_aligned.pdb -o ../molecules/D4BMM7_aligned -p', returncode=0)

In [4]:
# Prepare LNnT
#url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/conformers/009604C600000001/SDF?response_type=save&response_basename=Conformer3D_COMPOUND_CID_9831622'
#command = f'wget {url} -O {MOL}/LNnT.sdf'
#subprocess.run(command, shell=True, capture_output=False)
# SDF -> PDBQT
command = f'mk_prepare_ligand.py -i {MOL}/LNnT.sdf -o {MOL}/LNnT.pdbqt'
subprocess.run(command, shell=True, capture_output=False)

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


CompletedProcess(args='mk_prepare_ligand.py -i ../molecules/LNnT.sdf -o ../molecules/LNnT.pdbqt', returncode=0)

In [3]:
def read_mol(file_path: str) -> str:

    with open(file_path, "r") as f:
        pdbqt = f.readlines()
    
    # Extract only the lines starting with ATOM or HETATM
    pdb_lines = [line for line in pdbqt if line.startswith(("ATOM", "HETATM", "MODEL", "ENDMDL"))]
    pdb_str = "".join(pdb_lines)

    return pdb_str

def box2pdb(center: list, size: list, spacing = 1) -> str:
    lengths = [s * spacing for s in size]
    mins = [c - l / 2 for c, l in zip(center, lengths)]
    maxs = [c + l / 2 for c, l in zip(center, lengths)]
    corners = [
        (mins[0], mins[1], mins[2]),
        (maxs[0], mins[1], mins[2]),
        (maxs[0], maxs[1], mins[2]),
        (mins[0], maxs[1], mins[2]),
        (mins[0], mins[1], maxs[2]),
        (maxs[0], mins[1], maxs[2]),
        (maxs[0], maxs[1], maxs[2]),
        (mins[0], maxs[1], maxs[2]),
    ]

    box_pdb = ''
    for i, (x, y, z) in enumerate(corners, 1):
        box_pdb += f'ATOM      {i}   Ne BOX X   {i}    {x:8.3f}{y:8.3f}{z:8.3f}  1.00 10.00          Ne\n'

    box_pdb += '''
    CONECT    1    2
    CONECT    1    4
    CONECT    1    5
    CONECT    2    3
    CONECT    2    6
    CONECT    3    4
    CONECT    3    7
    CONECT    4    8
    CONECT    5    6
    CONECT    5    8
    CONECT    6    7
    CONECT    7    8
    '''
    return box_pdb

In [6]:
# Read pdbqt files
ligand = read_mol(f'{MOL}/ligand.pdbqt')
infantis = read_mol(f'{MOL}/8ibt.pdbqt')
breve = read_mol(f'{MOL}/{breveID}_aligned.pdbqt')

# Create docking grid 
center = [76, -29, -59]
size = [20, 10, 12]
box = box2pdb(center, size)

# Visualize complex + grid
view = py3Dmol.view(width=500, height=500)
view.addModel(infantis, 'pdb')
view.setStyle({'cartoon': {'color':'cyan', 'opacity': 0.7}})
view.addModel(breve, 'pdb')
view.setStyle({'model':-1}, {'cartoon': {'color':'red', 'opacity': 0.7}})
view.addModel(box, 'pdb')
view.addStyle({'elem': 'Ne'}, {'stick': {}})
view.addModel(ligand, 'pdb')
view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# Run docking LNT
receptor_file = f'{MOL}/{breveID}_aligned.pdbqt'
ligand_file = f'{MOL}/ligand.pdbqt'
vina = AutoDockVina(receptor_file, ligand_file)
exhaustiveness = 30
num_poses = 20
vina.dock(
    center=center, 
    box_size=size, 
    exhaustiveness=exhaustiveness, 
    n_poses=num_poses, 
    output_name='breve_output'
    ) 

Docking starting...
Docking finished. Results saved to breve_output.pdbqt.
Computing Vina grid ... done.
Performing docking (random seed: -372636941) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -4.505          0          0
   2       -4.123      1.716      4.024
   3       -2.867      1.539      2.533
   4       -2.717      2.276      10.58
   5       -1.992       2.21      4.167
   6       -1.874      1.474      2.383
   7       -1.541       2.12      10.36
   8      -0.8538      2.222      4.796
   9      -0.7977      2.564      5.411
  10      -0.7199      2.106      10.11
  11      -0.7106      2.928      5.598
  12     -0.04869      2.089      10.56
  13      0.01607      1.983      4.666
  14      0.06516      2.675      11.22
  1

array([-4.505, -4.123, -2.867, -2.717, -1.992, -1.874, -1.541])

In [7]:
# Run docking LNnT
receptor_file = f'{MOL}/{breveID}_aligned.pdbqt'
ligand_file = f'{MOL}/LNnT.pdbqt'
vina = AutoDockVina(receptor_file, ligand_file)
exhaustiveness = 30
num_poses = 20
vina.dock(
    center=center, 
    box_size=size, 
    exhaustiveness=exhaustiveness, 
    n_poses=num_poses, 
    output_name='LNnT_output'
    ) 

Computing Vina grid ... done.
Docking starting...
Performing docking (random seed: 1777235785) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -4.548          0          0
   2       -4.481      1.801      10.44
   3       -3.814      1.688      10.34
   4       -3.753      1.781      10.34
   5       -3.651       1.89      10.62
   6       -3.253      1.779      2.986
   7       -3.038      1.995      10.83
   8        -2.22      1.718      10.44
   9       -1.936      1.894      10.26
  10       -1.042      2.162      3.673
  11      -0.9353      2.082      3.908
  12      -0.8796      1.892      10.04
  13      -0.8321       2.33       10.4
  14      -0.7741      1.966      3.502
  15       -0.766      2.076      10.43
  16      -0.3673 

array([-4.548, -4.481, -3.814, -3.753, -3.651, -3.253, -3.038, -2.22 ,
       -1.936])

In [8]:
# Report results
vina.report()

Calculating RMSD...
Calculating energy...


{'rmsd': array([2.28814, 2.87152, 2.96226, 2.63649, 2.2365 , 2.48588, 2.42707,
        2.28914, 3.30902]),
 'energy': array([-4.548, -4.481, -3.814, -3.753, -3.651, -3.253, -3.038, -2.22 ,
        -1.936])}

In [13]:
# Read pdbqt files
ligand = read_mol(f'{MOL}/ligand.pdbqt')
infantis = read_mol(f'{MOL}/8ibt.pdbqt')
breve = read_mol(f'{MOL}/{breveID}_aligned.pdbqt')

# Create docking grid 
box = box2pdb(center, size)

# Visualize complex + grid + poses
def slider(pose: int):
    print(f'Pose {pose}')
    poses_lnt = read_mol(f'breve_output.pdbqt')
    pose_lnt = poses_lnt.split('ENDMDL\n')[:-1][pose - 1]
    poses_lnnt = read_mol(f'LNnT_output.pdbqt')
    pose_lnnt = poses_lnnt.split('ENDMDL\n')[:-1][pose - 1]
    view = py3Dmol.view(width=500, height=500)
    view.addModel(box, 'pdb')
    view.addStyle({'elem': 'Ne'}, {'stick': {}})
    #view.addModel(ligand, 'pdb')
    #view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
    view.zoomTo()
    view.addModel(breve, 'pdb')
    view.setStyle({'model':-1}, {'cartoon': {'color':'red', 'opacity': 0.7}})
    #view.addStyle({'and': [
    #    {'not': {'atom': ['N', 'C', 'O', 'CA']}},
    #    {'not': {'resn': 'HOH'}}  # Optional: exclude water
    #]}, {'stick': {'color':'red', 'opacity': 0.7}})
    view.addModel(infantis, 'pdb')
    view.setStyle({'model':-1}, {'cartoon': {'color':'cyan', 'opacity': 0.7}})
    view.addModel(pose_lnt, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'pinkCarbon'}})
    view.addModel(pose_lnnt, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'yellowCarbon'}})
    view.show()
    

interact(slider, pose=IntSlider(min=1, max=num_poses - 1, step=1))

interactive(children=(IntSlider(value=1, description='pose', max=19, min=1), Output()), _dom_classes=('widget-…

<function __main__.slider(pose: int)>